In [1]:
#!pip install xlsxwriter

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import xlsxwriter

In [3]:
df = pd.read_excel('CBS_CRM.xlsx', skiprows=1)

In [4]:
df.drop('Unnamed: 15', axis=1, inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   QTDE              204 non-null    float64       
 1   DATA 1º CONTATO   1672 non-null   object        
 2   NOME COMPLETO     1670 non-null   object        
 3   IDADE             1428 non-null   object        
 4   TELEFONE          1591 non-null   object        
 5   STATUS            1668 non-null   object        
 6   VIA               1668 non-null   object        
 7   ESTILOS           1664 non-null   object        
 8   DATA AGENDAMENTO  223 non-null    datetime64[ns]
 9   HORÁRIO           1301 non-null   object        
 10  PROFESSOR         1373 non-null   object        
 11  COMPARECEU        1650 non-null   object        
 12  REAGENDAMENTO     37 non-null     object        
 13  SITUAÇÃO          1640 non-null   object        
 14  OBSERVAÇÕES       838 no

In [6]:
print(df.isnull().sum())

QTDE                1470
DATA 1º CONTATO        2
NOME COMPLETO          4
IDADE                246
TELEFONE              83
STATUS                 6
VIA                    6
ESTILOS               10
DATA AGENDAMENTO    1451
HORÁRIO              373
PROFESSOR            301
COMPARECEU            24
REAGENDAMENTO       1637
SITUAÇÃO              34
OBSERVAÇÕES          836
dtype: int64


In [7]:
df.head(5)

,QTDE,DATA 1º CONTATO,NOME COMPLETO,IDADE,TELEFONE,STATUS,VIA,ESTILOS,DATA AGENDAMENTO,HORÁRIO,PROFESSOR,COMPARECEU,REAGENDAMENTO,SITUAÇÃO,OBSERVAÇÕES
0,1.0,2022-01-07 00:00:00,Graziele,22,12 99614-0592,AULA EXPERIMENTAL,WHATSAPP,JAZZ FUNK,NaT,18h30,ALISSON,SIM,NaN,MATRICULADO,NaN
1,2.0,2022-01-07 00:00:00,Camila,ADULTO,11 94129-6714,INFORMAÇÕES,WHATSAPP,JAZZ,NaT,19H30,LIA,NÃO,NaN,SEMANA SOLIDÁRIA,NaN
2,3.0,2021-01-05 00:00:00,MAIARA GOUVEA,16,98841-5432,AULA EXPERIMENTAL,WHATS,K-POP,NaT,19H00,ALISSON,NÃO FICOU,NaN,NÃO RESPONDE,NaN
3,4.0,2021-01-05 00:00:00,ENIALE RAMOS QUINTINO (MÃE ELAINE),11,99124-9509,AULA EXPERIMENTAL,WHATS,JAZZ,NaT,17H00,LIA,MATRICULADO,NaN,MATRICULADO,NaN
4,5.0,2022-01-07 00:00:00,Bia,ADULTO,12 98129-7939,INFORMAÇÕES,WHATSAPP,MODALIDADES,NaT,-,-,NÃO,NaN,SEMANA SOLIDÁRIA,NaN


In [8]:
df['DATA 1º CONTATO'] = df['DATA 1º CONTATO'].astype(str) #converter em string para alterações

In [9]:
df.dropna(subset=['DATA 1º CONTATO'], inplace=True) #dropar nulos

In [10]:
df = df[df['DATA 1º CONTATO'].astype(str).str.replace(' |-', '', regex=True).ne('')] #corrigir valores que não estão no formato

In [11]:
df['DATA 1º CONTATO'] = df['DATA 1º CONTATO'].str.replace('223', '2023').str.replace('2012', '2021') #corrigindo inconsistencias

<ipython-input-11-01155615faa9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATA 1º CONTATO'] = df['DATA 1º CONTATO'].str.replace('223', '2023').str.replace('2012', '2021') #corrigindo inconsistencias


In [12]:
#df['DATA 1º CONTATO'].unique()

In [13]:
df = df[pd.to_datetime(df['DATA 1º CONTATO'], errors='coerce').notna()] # dropar nulos
df['DATA 1º CONTATO'] = pd.to_datetime(df['DATA 1º CONTATO'], errors='coerce').dt.date  #converter em formato date

In [14]:
df.dropna(subset=['NOME COMPLETO'], inplace=True)  # Excluir valores nulos
df['NOME COMPLETO'] = df['NOME COMPLETO'].str.replace(r'[^\w\s]', '', regex=True)  # Remover caracteres especiais
df['NOME COMPLETO'] = df['NOME COMPLETO'].str.strip()  # Remover espaços em branco
df['NOME COMPLETO'] = df['NOME COMPLETO'].str.upper()

In [15]:
# Converter valores numéricos e '%-%' para float, e strings para NaN
df['IDADE'] = df['IDADE'].apply(lambda x: float(x) if isinstance(x, (int, float)) else
                                      (float(x.split('-')) if isinstance(x, str) and '-' in x else np.nan))

# Preencher valores NaN com a média da coluna
df['IDADE'].fillna(df['IDADE'].mean(), inplace=True)

# Alterar para tipo inteiro
df['IDADE'] = df['IDADE'].astype(int)

In [16]:
# Crie a coluna 'FAIXA ETARIA'
df['FAIXA ETARIA'] = df['IDADE'].apply(lambda x:
    'CRIANÇA' if x < 10 else
    'ADOLESCENTE' if x <= 15 else
    'ADULTO'
)

In [17]:
# Reordena as colunas para que 'FAIXA ETARIA' fique do lado direito de 'IDADE'
df = df[['DATA 1º CONTATO', 'NOME COMPLETO', 'IDADE', 'FAIXA ETARIA', 'TELEFONE', 'STATUS', 'VIA', 'ESTILOS', 'DATA AGENDAMENTO', 'HORÁRIO', 'PROFESSOR', 'COMPARECEU', 'REAGENDAMENTO', 'SITUAÇÃO', 'OBSERVAÇÕES']]

In [18]:
df.head()

,DATA 1º CONTATO,NOME COMPLETO,IDADE,FAIXA ETARIA,TELEFONE,STATUS,VIA,ESTILOS,DATA AGENDAMENTO,HORÁRIO,PROFESSOR,COMPARECEU,REAGENDAMENTO,SITUAÇÃO,OBSERVAÇÕES
0,2022-01-07,GRAZIELE,22,ADULTO,12 99614-0592,AULA EXPERIMENTAL,WHATSAPP,JAZZ FUNK,NaT,18h30,ALISSON,SIM,NaN,MATRICULADO,NaN
1,2022-01-07,CAMILA,19,ADULTO,11 94129-6714,INFORMAÇÕES,WHATSAPP,JAZZ,NaT,19H30,LIA,NÃO,NaN,SEMANA SOLIDÁRIA,NaN
2,2021-01-05,MAIARA GOUVEA,16,ADULTO,98841-5432,AULA EXPERIMENTAL,WHATS,K-POP,NaT,19H00,ALISSON,NÃO FICOU,NaN,NÃO RESPONDE,NaN
3,2021-01-05,ENIALE RAMOS QUINTINO MÃE ELAINE,11,ADOLESCENTE,99124-9509,AULA EXPERIMENTAL,WHATS,JAZZ,NaT,17H00,LIA,MATRICULADO,NaN,MATRICULADO,NaN
4,2022-01-07,BIA,19,ADULTO,12 98129-7939,INFORMAÇÕES,WHATSAPP,MODALIDADES,NaT,-,-,NÃO,NaN,SEMANA SOLIDÁRIA,NaN


In [19]:
# Ignore os caracteres especiais
df['TELEFONE'] = df['TELEFONE'].str.replace(r'[^\d]', '')

# Padronize os valores nulos com '0000-0000'
df['TELEFONE'].fillna('0000-0000', inplace=True)

In [20]:
df['STATUS'].unique()

array(['AULA EXPERIMENTAL', 'INFORMAÇÕES', 'AULA ESPERIMENTAL',
       'INFORMAÇÕES ', 'CAPTAÇÃO', 'COREOGRAFIA', 'AULA AVULSA',
       'INFORMACOES', 'INFORMAÇÃO', 'INFORMAÇOES', nan], dtype=object)

In [21]:
# Padronize 'INFORMAÇÕES'
df['STATUS'] = df['STATUS'].str.replace('INFORMAÇÕES ', 'INFORMAÇÕES')
df['STATUS'] = df['STATUS'].str.replace('INFORMACOES', 'INFORMAÇÕES')
df['STATUS'] = df['STATUS'].str.replace('INFORMAÇÃO', 'INFORMAÇÕES')
df['STATUS'] = df['STATUS'].str.replace('INFORMAÇOES', 'INFORMAÇÕES')
df['STATUS'] = df['STATUS'].str.replace('INFORMAÇÕES', 'CAPTAÇÃO')
# Substitua 'AULA AVULSA' por 'EXPERIMENTAL'
df['STATUS'] = df['STATUS'].str.replace('AULA AVULSA', 'EXPERIMENTAL')
df['STATUS'] = df['STATUS'].str.replace('AULA EXPERIMENTAL', 'EXPERIMENTAL')

# Padronize 'AULA ESPERIMENTAL'
df['STATUS'] = df['STATUS'].str.replace('AULA ESPERIMENTAL', 'EXPERIMENTAL')
df['STATUS'] = df['STATUS'].str.replace('AULA AULA EXPERIMENTAL', 'EXPERIMENTAL')
df['STATUS'] = df['STATUS'].str.replace('COREOGRAFIA', 'EXPERIMENTAL')


# Excluir nulos
df = df.dropna(subset=['STATUS'])

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1665 entries, 0 to 1673
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATA 1º CONTATO   1665 non-null   object        
 1   NOME COMPLETO     1665 non-null   object        
 2   IDADE             1665 non-null   int64         
 3   FAIXA ETARIA      1665 non-null   object        
 4   TELEFONE          1665 non-null   object        
 5   STATUS            1665 non-null   object        
 6   VIA               1665 non-null   object        
 7   ESTILOS           1661 non-null   object        
 8   DATA AGENDAMENTO  222 non-null    datetime64[ns]
 9   HORÁRIO           1297 non-null   object        
 10  PROFESSOR         1370 non-null   object        
 11  COMPARECEU        1647 non-null   object        
 12  REAGENDAMENTO     37 non-null     object        
 13  SITUAÇÃO          1636 non-null   object        
 14  OBSERVAÇÕES       838 non-nul

In [23]:
df['VIA'].unique()

array(['WHATSAPP', 'WHATS', 'INSTAGRAM', 'TELEFONE', 'PESSOALMENTE',
       'PRESENCIAL', 'PRESENCIAL ', 'SITE', 'VALE AULA', 'FACEBOOK',
       'MESSENGER', 'MAE JÁ É ALUNA', 'ALUNA'], dtype=object)

In [24]:
#padrozniar
df['VIA'] = df['VIA'].str.replace('WHATSAPPAPP', 'WHATSAPP')
df['VIA'] = df['VIA'].str.replace('WHATSAPP', 'WHATSAPP')
df['VIA'] = df['VIA'].str.replace('TELEFONE', 'WHATSAPP')
df['VIA'] = df['VIA'].str.replace('PESSOALMENTE', 'PRESENCIAL')
df['VIA'] = df['VIA'].str.replace('VALE ', 'PRESENCIAL')
df['VIA'] = df['VIA'].str.replace('MAE JÁ É ALUNA', 'PRESENCIAL')
df['VIA'] = df['VIA'].str.replace('ALUNA', 'PRESENCIAL')
df['VIA'] = df['VIA'].str.replace('MESSENGER', 'FACEBOOK')
df['VIA'] = df['VIA'].str.replace('PRESENCIALAULA', 'PRESENCIAL')
df['VIA'] = df['VIA'].str.replace('PRESENCIAL ', 'PRESENCIAL')

In [25]:
# Padronização
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZ FUNK ', 'JAZZ FUNK')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZ FUNKINICIANTE', 'JAZZ FUNK')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZ FUNKINTERMEDIARIO', 'JAZZ FUNK')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZFUNK', 'JAZZ FUNK')


df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZ ', 'JAZZ')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZ INICIANTE', 'JAZZ')
df['ESTILOS'] = df['ESTILOS'].str.replace('Jazz', 'JAZZ')
df['ESTILOS'] = df['ESTILOS'].str.replace('Jazz iniciante', 'JAZZ')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZINICIANTE', 'JAZZ')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZ iniciante', 'JAZZ')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZiniciante', 'JAZZ')

df['ESTILOS'] = df['ESTILOS'].str.replace('AULA DE BREAK', 'DANÇAS URBANAS')
df['ESTILOS'] = df['ESTILOS'].str.replace('JAZZE D URBANAS', 'DANÇAS URBANAS')


In [26]:
df['ESTILOS'] = df['ESTILOS'].str.replace(r'[^\w\s]+', '', regex=True).str.replace('\s+', ' ', regex=True).str.strip()

In [27]:
df['ESTILOS'] = df['ESTILOS'].str.replace(r'BALLET.*|KPOPBALLET.*|BALLEYT', 'BALLET CLÁSSICO', regex=True)
df['ESTILOS'] = df['ESTILOS'].str.replace(r'KPOP.*|K POP INTERMEDIARIO|K POP INICIANTE|kpop|K Pop|Kpop|K POP', 'K-POP', regex=True)
df['ESTILOS'] = df['ESTILOS'].str.replace(r'SAPATEADO.*|Sapateado', 'SAPATEADO', regex=True)
df['ESTILOS'] = df['ESTILOS'].str.replace(r'CONTEMPORANEO.*', 'CONTEMPORÂNEO', regex=True)

In [28]:
# Contar a frequência de cada valor em ESTILO
value_counts = df['ESTILOS'].value_counts()

In [29]:
# Selecionar os 3 valores mais frequentes
top_3_values = value_counts.nlargest(3).index.tolist()

In [30]:
top_3_values

['DANÇAS URBANAS', 'JAZZ', 'JAZZFUNK']

In [31]:
# Padronizar os valores que não são "JAZZFUNK", "JAZZ", "K-POP", "SAPATEADO", "DANÇAS URBANAS", "CONTEMPORÂNEO" e "BALLET CLÁSSICO" e distribuir igualitariamente entre estes os 3 valores que mais aparecem
df['ESTILOS'] = np.where(df['ESTILOS'].isin(['JAZZFUNK', 'JAZZ', 'K-POP', 'SAPATEADO', 'DANÇAS URBANAS', 'CONTEMPORÂNEO', 'BALLET CLÁSSICO']),
                        df['ESTILOS'],
                        np.random.choice(top_3_values))

In [32]:
df['DATA AGENDAMENTO'] = df['DATA AGENDAMENTO'].astype(str)

In [33]:
# Preencha os valores vazios com "Não agendou"
df['DATA AGENDAMENTO'].fillna('Não agendou', inplace=True)


# Altere o ano de 2032 para 2023
df['DATA AGENDAMENTO'] = df['DATA AGENDAMENTO'].apply(lambda x: '2023-' + x[5:] if x and x.startswith('2032') else x)

In [34]:
# Renomear a coluna HORÁRIO para TURNO
df = df.rename(columns={'HORÁRIO': 'TURNO'})

In [35]:
# função que converte um horário em um turno do dia
def convert_horario(x):
    if isinstance(x, str):
        if '-' in x or x[:2].isalpha():
            return 'Unknown'
        else:
            hour = ''.join(filter(str.isdigit, x))
            if hour:
                hour = int(hour)
                if 6 <= hour <= 12:
                    return 'MANHA'
                elif 12 <= hour <= 18:
                    return 'TARDE'
                else:
                    return 'NOITE'
            else:
                return 'Unknown'
    else:
        return 'Unknown'

df['TURNO'] = df['TURNO'].apply(convert_horario)

In [36]:
# Substituição dos valores 'Unknown' pela moda
moda_turno = df['TURNO'].mode()[0]
df['TURNO'] = df['TURNO'].replace('Unknown', moda_turno)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1665 entries, 0 to 1673
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   DATA 1º CONTATO   1665 non-null   object
 1   NOME COMPLETO     1665 non-null   object
 2   IDADE             1665 non-null   int64 
 3   FAIXA ETARIA      1665 non-null   object
 4   TELEFONE          1665 non-null   object
 5   STATUS            1665 non-null   object
 6   VIA               1665 non-null   object
 7   ESTILOS           1665 non-null   object
 8   DATA AGENDAMENTO  1665 non-null   object
 9   TURNO             1665 non-null   object
 10  PROFESSOR         1370 non-null   object
 11  COMPARECEU        1647 non-null   object
 12  REAGENDAMENTO     37 non-null     object
 13  SITUAÇÃO          1636 non-null   object
 14  OBSERVAÇÕES       838 non-null    object
dtypes: int64(1), object(14)
memory usage: 208.1+ KB


In [38]:
df['PROFESSOR'].unique()

array(['ALISSON', 'LIA', '-', 'BIANCA', 'NICOLAS', 'FERNANDO', 'GIBELI',
       'PAOLA', nan, 'GABRIELA', 'LOUBACH', 'ALISSIN', 'ALISSON ', 'LIA ',
       'CAROL', 'ISABELLY', 'LARISSA', 'AMANDA', 'CICERO', 'ALINE',
       'ALLISON ', 'CÍCERO', 'NÃO', 'ALISSON/LARI', 'TODAS', 'NÃO ',
       'LARISSA / ALISSON', 'FELIPE'], dtype=object)

In [39]:
# Substituir variantes de nomes e preencher vazios ou inconsistentes
df['PROFESSOR'] = df['PROFESSOR'].str.replace(r'ALISSIN.*|ALISSON .*|ALLISON ', 'ALISSON', regex=True)
df['ESTILOS'] = df['ESTILOS'].str.replace('LIA ', 'LIA')

df['PROFESSOR'].fillna('NÃO INFORMADO', inplace=True)
df.loc[df['PROFESSOR'] == 'NÃO', 'PROFESSOR'] = 'NÃO INFORMADO'
df.loc[df['PROFESSOR'] == 'NÃO ', 'PROFESSOR'] = 'NÃO INFORMADO'
df.loc[df['PROFESSOR'] == '-', 'PROFESSOR'] = 'NÃO INFORMADO'
df.loc[df['PROFESSOR'] == 'TODAS', 'PROFESSOR'] = 'NÃO INFORMADO'
df.loc[df['PROFESSOR'] == 'LARISSA / ALISSON', 'PROFESSOR'] = 'NÃO INFORMADO'

In [40]:
df['COMPARECEU'].unique()

array(['SIM', 'NÃO', 'NÃO FICOU', 'MATRICULADO', 'SEM RETORNO',
       'NÃO VEIO', 'MATRICULADA', 'VEIO', '-', 'AVULSO', 'NÃO RESPONDE',
       'NÃO VEIO ', 'ENROLADA', nan, 'MATRICULADO ', 'NÃO VAI FICAR',
       'AVULSA', 'CANCELOU', 'ANO QUE VEM',
       datetime.datetime(2023, 7, 31, 0, 0),
       datetime.datetime(2023, 4, 14, 0, 0),
       datetime.datetime(2023, 4, 13, 0, 0),
       datetime.datetime(2023, 4, 24, 0, 0),
       datetime.datetime(2023, 4, 25, 0, 0),
       datetime.datetime(2023, 4, 27, 0, 0),
       datetime.datetime(2023, 4, 28, 0, 0),
       datetime.datetime(2023, 5, 3, 0, 0),
       datetime.datetime(2023, 4, 20, 0, 0),
       datetime.datetime(2023, 5, 2, 0, 0),
       datetime.datetime(2023, 5, 4, 0, 0),
       datetime.datetime(2023, 5, 5, 0, 0),
       datetime.datetime(2023, 5, 12, 0, 0),
       datetime.datetime(2023, 5, 8, 0, 0),
       datetime.datetime(2023, 5, 19, 0, 0),
       datetime.datetime(2023, 5, 17, 0, 0),
       datetime.datetime(2023, 5, 

In [41]:
# alterar para tipo string
df['COMPARECEU'] = df['COMPARECEU'].astype(str)

In [42]:
# Defina os valores que devem ser considerados como 'SIM'
valores_sim = ['SIM', 'MATRICULADO', 'MATRICULADO ', 'CONFIRMADA', 'MATRICULADA', 'VEIO', '18//05/23']

# Defina os valores que devem ser considerados como 'NÃO'
valores_nao = ['nan','SUMIU','TODAS','NÃO ', 'ANO QUE VEM', 'NÃO VAI FICAR', 'AVULSA', 'CANCELOU', '-', 'AVULSO', 'NÃO RESPONDE', 'NÃO VEIO ', 'ENROLADA', 'SEM RETORNO', 'NÃO VEIO', 'NÃO', 'NÃO FICOU']

# Converter valores de data para objetos pd.Timestamp
df['COMPARECEU'] = pd.to_datetime(df['COMPARECEU'], errors='ignore')

# Substitua os valores que conhenham 2023 por 'SIM', pois significa que marcou uma data
df['COMPARECEU'] = np.where(df['COMPARECEU'].str.startswith('2023'), 'SIM', df['COMPARECEU'])

# Substitua os valores que estão em 'valores_sim' por 'SIM'
df['COMPARECEU'] = np.where(df['COMPARECEU'].isin(valores_sim), 'SIM', df['COMPARECEU'])

# Substitua os valores que estão em 'valores_nao' por 'NÃO'
df['COMPARECEU'] = np.where(df['COMPARECEU'].isin(valores_nao), 'NÃO', df['COMPARECEU'])

<ipython-input-42-ccb656d228fe>:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['COMPARECEU'] = pd.to_datetime(df['COMPARECEU'], errors='ignore')


In [43]:
df['REAGENDAMENTO'].unique()

array([nan, 'SIM', 'NÃO'], dtype=object)

In [44]:
#preencher os valores nulos
df['REAGENDAMENTO'].fillna('NÃO', inplace=True)

In [45]:
df['SITUAÇÃO'].unique()

array(['MATRICULADO', 'SEMANA SOLIDÁRIA', 'NÃO RESPONDE', 'QUER BOLSA',
       'DCTOS BOLSA', 'OUTRA ESCOLA', 'CONFIRMADO',
       'ESTAVA SEM DINHEIRO MARÇO', 'PORTA ABERTA', 'MATRICULADA',
       'IRMÃ DE ALUNA', 'ENROLADA, NÃO RESPONDE', 'QUER GINASTICA', '-',
       'NÃO QUER MAIS', 'COMPARECEU 10/02', '2º SEMESTRE', 'PARATY',
       'SEM DINHEIRO', 'SEM RESPOSTA', 'VAI NO  SAPATEADO',
       'NÃO VEIO 23/02', 'NÃO ATENDE', 'QUER JAZZ', 'ENTRE JAZZ E STREET',
       'FICOU SEM CARRO EM 01/03', 'ASSINAR CONTRATO',
       'NÃO VEIO 23/03 e 25/02', 'NAÕ VEIO 25/02', 'INFORMAÇÕES',
       'COMPARECEU 22/02', 'AGENDADO', 'ESPERAR MAIS',
       'COMPLICADA DE HORÁRIO', 'CHOROU NA SALA',
       'COMPARECEU CRISE ANSIEDADE', 'ENROLADA', 'COMPARECEU 03/03',
       'SEM ENERGIA', 'NÃO VEIO 02/03', 'AVULSO',
       'COMPARECEU 04/03 - VAI FAZER JAZZ 18H',
       'AMIGA MATEUS Ñ CONSEGUIU VIR', 'MORA FORA',
       'VEIO ACOMPANHAR AMIGO', 'REMARCADO VÁRIAS VEZES - ENROLADA',
       'CANCELOU',

In [46]:
# Substitui os valores na coluna 'SITUAÇÃO' com 'CONTRATADO' se contiver 'CONTRAT' ou 'MATRICULA' (case-insensitive)
df['SITUAÇÃO'] = np.where(df['SITUAÇÃO'].str.contains('CONTRAT|MATRICULA', case=False), 'MATRICULADO', df['SITUAÇÃO'])
df['SITUAÇÃO'] = np.where(df['SITUAÇÃO'].str.contains('NÃO'), 'DESISTIU', df['SITUAÇÃO'])

In [47]:
#with pd.ExcelWriter('CRM.xlsx', engine='xlsxwriter') as writer:
#    df.to_excel(writer, sheet_name='CRM', index=False)
    #df_pc.to_excel(writer, sheet_name='Plano de contas', index=False)

In [48]:
print(df.isnull().sum())

DATA 1º CONTATO       0
NOME COMPLETO         0
IDADE                 0
FAIXA ETARIA          0
TELEFONE              0
STATUS                0
VIA                   0
ESTILOS               0
DATA AGENDAMENTO      0
TURNO                 0
PROFESSOR             0
COMPARECEU            0
REAGENDAMENTO         0
SITUAÇÃO              0
OBSERVAÇÕES         827
dtype: int64


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1665 entries, 0 to 1673
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   DATA 1º CONTATO   1665 non-null   object
 1   NOME COMPLETO     1665 non-null   object
 2   IDADE             1665 non-null   int64 
 3   FAIXA ETARIA      1665 non-null   object
 4   TELEFONE          1665 non-null   object
 5   STATUS            1665 non-null   object
 6   VIA               1665 non-null   object
 7   ESTILOS           1665 non-null   object
 8   DATA AGENDAMENTO  1665 non-null   object
 9   TURNO             1665 non-null   object
 10  PROFESSOR         1665 non-null   object
 11  COMPARECEU        1665 non-null   object
 12  REAGENDAMENTO     1665 non-null   object
 13  SITUAÇÃO          1665 non-null   object
 14  OBSERVAÇÕES       838 non-null    object
dtypes: int64(1), object(14)
memory usage: 272.7+ KB


In [49]:
df.head(5)

,DATA 1º CONTATO,NOME COMPLETO,IDADE,FAIXA ETARIA,TELEFONE,STATUS,VIA,ESTILOS,DATA AGENDAMENTO,TURNO,PROFESSOR,COMPARECEU,REAGENDAMENTO,SITUAÇÃO,OBSERVAÇÕES
0,2022-01-07,GRAZIELE,22,ADULTO,12 99614-0592,EXPERIMENTAL,WHATSAPP,JAZZFUNK,NaT,NOITE,ALISSON,SIM,NÃO,MATRICULADO,NaN
1,2022-01-07,CAMILA,19,ADULTO,11 94129-6714,CAPTAÇÃO,WHATSAPP,JAZZ,NaT,NOITE,LIA,NÃO,NÃO,SEMANA SOLIDÁRIA,NaN
2,2021-01-05,MAIARA GOUVEA,16,ADULTO,98841-5432,EXPERIMENTAL,WHATS,K-POP,NaT,NOITE,ALISSON,NÃO,NÃO,DESISTIU,NaN
3,2021-01-05,ENIALE RAMOS QUINTINO MÃE ELAINE,11,ADOLESCENTE,99124-9509,EXPERIMENTAL,WHATS,JAZZ,NaT,NOITE,LIA,SIM,NÃO,MATRICULADO,NaN
4,2022-01-07,BIA,19,ADULTO,12 98129-7939,CAPTAÇÃO,WHATSAPP,JAZZ,NaT,NOITE,NÃO INFORMADO,NÃO,NÃO,SEMANA SOLIDÁRIA,NaN


In [50]:
# Exportar um arquivo xlsx com as formatações/filtros
# Configurações de formatação
header_format = {
    'bold': True,
    'font_size': 12,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': '#D3D3D3',
    'border': 1
}

cell_format = {
    'font_size': 11,
    'align': 'center',
    'valign': 'vcenter',
    'border': 1
}

# Crie o objeto ExcelWriter
with pd.ExcelWriter('CRM.xlsx', engine='xlsxwriter') as writer:
    # Exporte o DataFrame para o arquivo xlsx
    df.to_excel(writer, sheet_name='CRM', index=False)

    # Formate a planilha
    workbook = writer.book
    worksheet = writer.sheets['CRM']

    # Adicione o filtro ao cabeçalho
    worksheet.autofilter(0, 0, 0, len(df.columns) - 1)

    # Formatação do cabeçalho
    header_format_obj = workbook.add_format(header_format)
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, header_format_obj)

    # Formatação das células
    cell_format_obj = workbook.add_format(cell_format)
    date_format_obj = workbook.add_format({'num_format': 'yyyy-mm-dd', 'border': 1})

    for row_num, row_data in enumerate(df.values):
        for col_num, value in enumerate(row_data):
            if pd.isnull(value):
                worksheet.write(row_num + 1, col_num, '', cell_format_obj)
            elif col_num == 0:
                worksheet.write(row_num + 1, col_num, value, date_format_obj)
            else:
                worksheet.write(row_num + 1, col_num, value, cell_format_obj)

    # Ajuste a largura das colunas
    for col_num, col_width in enumerate([20, 20, 20]):
        worksheet.set_column(col_num, col_num, col_width)

    # Descomente a linha abaixo se você tiver outro DataFrame para exportar
    # df_pc.to_excel(writer, sheet_name='Plano de contas', index=False)